# Medical Image Binary Classification with Convolutional Neural Networks

This Python code is designed to create and train a Convolutional Neural Network (CNN) for binary classification of medical images. The dataset contains chest X-ray images categorized into two classes: NORMAL and PNEUMONIA.


1. **Import Libraries**:

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
import numpy as np

2. **Set Dataset Paths, Image Dimensions and Batch Size**:

In [2]:
# Set the paths to your dataset
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
val_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'

# Image dimensions and batch size
image_size = (224, 224)
batch_size = 32

3. **Data Augmentation**:

In [3]:
# Data augmentation for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

4. **Data Preprocessing**:

In [4]:
# Preprocess and augment the training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.


In [5]:
# Preprocess the test and validation data
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 624 images belonging to 2 classes.


In [6]:
val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 16 images belonging to 2 classes.


5. **Create a CNN Model**:

In [7]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(base_model.input, x)

58889256/58889256 [==============================] - 0s 0us/step


6. **Compile & Train the Model**:

In [8]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
163/163 [==============================] - 158s 867ms/step - loss: 0.3691 - accuracy: 0.8788 - val_loss: 0.4942 - val_accuracy: 0.7500
Epoch 2/10
163/163 [==============================] - 98s 600ms/step - loss: 0.2029 - accuracy: 0.9151 - val_loss: 0.5119 - val_accuracy: 0.7500
Epoch 3/10
163/163 [==============================] - 98s 598ms/step - loss: 0.1870 - accuracy: 0.9229 - val_loss: 0.5637 - val_accuracy: 0.7500
Epoch 4/10
163/163 [==============================] - 98s 602ms/step - loss: 0.1796 - accuracy: 0.9291 - val_loss: 0.4176 - val_accuracy: 0.8125
Epoch 5/10
163/163 [==============================] - 99s 603ms/step - loss: 0.1767 - accuracy: 0.9270 - val_loss: 0.3019 - val_accuracy: 0.7500
Epoch 6/10
163/163 [==============================] - 98s 601ms/step - loss: 0.1796 - accuracy: 0.9302 - val_loss: 0.4371 - val_accuracy: 0.8125
Epoch 7/10
163/163 [==============================] - 98s 599ms/step - loss: 0.1630 - accuracy: 0.9344 - val_loss: 0.2269 - val_a

7. **Evaluate & save the Model**:

In [9]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

20/20 [==============================] - 10s 503ms/step - loss: 0.2908 - accuracy: 0.9054
Test accuracy: 90.54%


In [11]:
# Save the model
model.save('/kaggle/working/cnn_model.h5')

8. **Test prediction**:

In [16]:
# Load the trained model
model = tf.keras.models.load_model('/kaggle/working/cnn_model.h5')

# Load an example image for prediction
image_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg'
img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

# Make prediction
predictions = model.predict(img)

# Interpret the prediction
if predictions[0] < 0.5:
    print("The image is NORMAL.")
else:
    print("The image indicates PNEUMONIA.")


1/1 [==============================] - 0s 144ms/step
The image indicates PNEUMONIA.
